## Nan 처리하기

- Nan 을 처리할때, 결국 중요시 되는 feature들은 보험사기자들이 어떤 feature를 가지고 있는지 체크해봐야함. 일반고객의 수 > 보험사기자 수 이기 때문에 ==> 일반고객들의 null값을 살펴보기보다는 보험사기자들의 null값들이 어떻게 되어있는지 살펴본 후, 어떻게 빈칸을 처리할 것인지 판단하려고 함.

In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
merged_df_except_family = pd.read_csv('./Data/merged_df_except_family.csv', encoding='utf-16')

### 전처리 대상 Columns

In [5]:
# Null 값이 포함된 Column 찾기
columns = merged_df_except_family.columns
null_in_columns = []
for column in columns:
    result = merged_df_except_family[column].isnull().any()
    if result == True:
        null_in_columns.append(column)
print("< 전처리 대상 컬럼 리스트 >")
print(null_in_columns)
print(" ")
print("< 전처리 대상 컬럼 수 >")
print(len(null_in_columns))

< 전처리 대상 컬럼 리스트 >
['SIU_CUST_YN', 'RESI_TYPE_CODE', 'CUST_RGST', 'CTPR', 'OCCP_GRP_1', 'OCCP_GRP_2', 'TOTALPREM', 'MINCRDT', 'MAXCRDT', 'WEDD_YN', 'MATE_OCCP_GRP_1', 'MATE_OCCP_GRP_2', 'CHLD_CNT', 'LTBN_CHLD_AGE', 'MAX_PAYM_YM', 'MAX_PRM', 'CUST_INCM', 'JPBASE_HSHD_INCM', 'EXTN_YM', 'LAPS_YM', 'DISTANCE', 'ACCI_OCCP_GRP1', 'ACCI_OCCP_GRP2', 'HOSP_OTPA_STDT', 'HOSP_OTPA_ENDT', 'HOUSE_HOSP_DIST', 'HOSP_CODE', 'ACCI_HOSP_ADDR', 'HOSP_SPEC_DVSN', 'PAYM_DATE', 'SELF_CHAM', 'NON_PAY', 'TAMT_SFCA', 'PATT_CHRG_TOTA', 'DSCT_AMT', 'COUNT_TRMT_ITEM', 'DCAF_CMPS_XCPA', 'BEFO_JOB', 'EDGB']
 
< 전처리 대상 컬럼 수 >
39


### target 값을 제외한 38개의 컬럼
* SIU_CUST_YN : 보험사기자 구분 (X)
* RESI_TYPE_CODE : 주택유형 ==> 주택가격이 0이면 null ==> 없음으로 처리가능
* CUST_RGST : 고객등록일
* CTPR : 거주 주소
* OCCP_GRP_1 : 직업분류코드1 ==> 실업자의 경우 직업이 없을 수 있음
* OCCP_GRP_2 : 직업분류코드2 ==> 위와 동일
* TOTALPREM : 납입총보험료
* MINCRDT : 최소신용등급 ==> 미확인시 6등급 ==> 6등급으로 처리
* MAXCRDT : 최대신용등급 ==> 미확인시 6등급 ==> 6등급으로 처리
* WEDD_YN : 결혼여부
* MATE_OCCP_GRP_1 : 배우자직업코드1 ==> 결혼을 하지 않았으면 null
* MATE_OCCP_GRP_2 : 배우자직업코드2 ==> 위와동일
* CHLD_CNT : 자녀수 
* LTBN_CHLD_AGE : 막내자녀연령
* MAX_PAYM_YM : 최대보험료연월
* MAX_PRM : 최대보험료
* CUST_INCM : 고객의 추정소득
* JPBASE_HSHD_INCM : 가구의 추정소득2
* EXTN_YM : 소멸년월 (없으면 null) ==> 이건 어떻게 처리?
* LAPS_YM : 실효년월 (없으면 null) ==> 이건 어떻게 처리?
* DISTANCE : FP와 고객거주지의 거리 ==> 거주주소가 없으므로 거리도 나오지 않음
* ACCI_OCCP_GRP1 : 직업그룹코드1
* ACCI_OCCP_GRP2 : 직업그룹코드2
* HOSP_OTPA_STDT : 입원/통원시작일자
* HOSP_OTPA_ENDT : 입원/통원종료일자
* HOUSE_HOSP_DIST : 고객 거주지와 병원까지 거리
* HOSP_CODE : 병원코드
* ACCI_HOSP_ADDR : 병원지역(시도)
* HOSP_SPEC_DVSN : 병원종별구분
* PAYM_DATE : 지급일자
* SELF_CHAM : 본인부담금
* NON_PAY : 비급여
* TAMT_SFCA : 전액본인부담금
* PATT_CHRG_TOTA : 환자부담총액
* DSCT_AMT : 병원에서 할인해준 금액
* COUNT_TRMT_ITEM : 진료과목갯수
* DCAF_CMPS_XCPA : 실손상품에서 보상제외금액을 의미
* BEFO_JOB : FP 입사전 직업
* EDGB : FP 학력

* "#" 값의경우 정확한 내용을 알 수 없음으로 해석하면 됨